In [ ]:
#!pip install scikit-learn numpy torch scipy monai torchmetrics
from sklearn.utils import column_or_1d
from sklearn.preprocessing import label_binarize
import numpy as np
import torch
from scipy.stats import norm
from monai.metrics import DiceMetric
from monai.transforms import AsDiscrete
from scipy.integrate import quad
from scipy.interpolate import interp1d
from monai.transforms import CropForeground

from torchmetrics.classification import MulticlassCalibrationError
import nibabel as nib


'''
Prepare the annotations and results to be processed
'''

def format_normalisation (result_file, prob_file, GT1_file, GT2_file, GT3_file, ct_file) :
    """ 
    Adapts CT scans results to the right format (slice, X, Y)
    """
    bin_pred = nib.load(result_file).get_fdata().astype(np.uint8) 

    #(slices, X, Y)
    bin_pred = bin_pred.transpose(2, 0, 1)

    prob_data = nib.load(prob_file).get_fdata()

    #(classes, slices, X, Y)
    prob_pred = prob_data.transpose(0, 3, 1, 2)

    pancreas_conf = prob_pred[0] 
    kidney_conf = prob_pred[1]
    liver_conf = prob_pred[2]
    results = [bin_pred, pancreas_conf, kidney_conf, liver_conf]

    #Pancreas ground truth
    gt1 = nib.load(GT1_file).get_fdata()

    #(slices, X, Y)
    gt1 = gt1.transpose(2, 0, 1)
    
    #Kidney ground truth
    gt2 = nib.load(GT2_file).get_fdata()

    #(slices, X, Y)
    gt2 = gt2.transpose(2, 0, 1)
    
    #Liver ground truth
    gt3 = nib.load(GT3_file).get_fdata()

    #(slices, X, Y)
    gt3 = gt3.transpose(2, 0, 1)

    annotations = [gt1, gt2, gt3]

    #CT scan, unlabeled with the three organs
    ct_image = nib.load(ct_file).get_fdata()

    #(slices, X, Y)
    ct_image = ct_image.transpose(2, 0, 1)



    return ct_image, annotations, results

def preprocess_results(ct_image, annotations, results):
    """
    Preprocess the images, predictions and annotations in order to be evaluated.
    It crops the foreground and applies the same crop to the rest of matrices.
    This is done to save some memory and work with smaller matrices.
    
    ct_image: CT images of shape (slices, X, Y)
    annotations: list containing the three ground truths [gt1, gt2, gt3]
                 each gt has the following values: 1: pancreas, 2: kidney, 3: liver
                 each gt has the following shape (slices, X, Y)
    results: list containing the results [binarized prediction, 
                                          pancreas_confidence,
                                          kidney_confidence,
                                          liver_confidence
                                         ]
            the binarized prediction the following values: 1: pancreas, 2: kidney, 3: liver
            each confidence has probabilistic values that range from 0 to 1
     
    @output cropped_annotations, cropped_results[0], cropped_results[1:]
  
    """
    
    # Define the CropForeground transform
    cropper = CropForeground(select_fn=lambda x: x > 0)  # Assuming non-zero voxels are foreground

    # Compute the cropping box based on the CT image
    box_start, box_end = cropper.compute_bounding_box(ct_image)
    
    # Apply the cropping box to all annotations
    cropped_annotations = [annotation[..., box_start[0]:box_end[0], box_start[1]:box_end[1]] for annotation in annotations]
    cropped_results = [result[..., box_start[0]:box_end[0], box_start[1]:box_end[1]] for result in results]

    return cropped_annotations, cropped_results[0], cropped_results[1:]


'''
Dice Score Evaluation
'''

def consensus_dice_score(groundtruth, bin_pred, prob_pred):
    """
    Computes an average of dice score for consensus areas only.
    
    groundtruth: numpy stack list containing the three ground truths [gt1, gt2, gt3]
                 each gt has the following values: 1: pancreas, 2: kidney, 3: liver
                    (3, slices, X, Y)
    bin_pred: binarized prediction matrix containing values: {0,1,2,3}
    prob_pred: probability prediction matrix, shape: (3, slices, X, Y), the three being
                a probability matrix per each class
     
    @output dice_scores, confidence
  
    """
    
    # Transform probability predictions to one-hot encoding by taking the argmax
    prediction_onehot = AsDiscrete(to_onehot=4)(torch.from_numpy(np.expand_dims(bin_pred, axis=0)))[1:].astype(np.uint8)
    
    # Split ground truth into separate organs and calculate consensus
    organs =  {1: 'panc', 2: 'kidn', 3: 'livr'}
    consensus = {}
    dissensus = {}

    for organ_val, organ_name in organs.items():
        # Get the ground truth for the current organ
        organ_gt = (groundtruth == organ_val).astype(np.uint8)
        organ_bck = (groundtruth != organ_val).astype(np.uint8)
        
        # Calculate consensus regions (all annotators agree)
        consensus[organ_name] = np.logical_and.reduce(organ_gt, axis=0).astype(np.uint8)
        consensus[f"{organ_name}_bck"] = np.logical_and.reduce(organ_bck, axis=0).astype(np.uint8)
        
        # Calculate dissensus regions (where both background and foreground are 0)
        dissensus[organ_name] = np.logical_and(consensus[organ_name] == 0, 
                                               consensus[f"{organ_name}_bck"]== 0).astype(np.uint8)
    
    # Mask the predictions and ground truth with the consensus areas
    predictions = {}
    groundtruth_consensus = {}
    confidence = {}

    for organ_val, organ_name in organs.items():
        # Apply the dissensus mask to exclude non-consensus areas
        filtered_prediction = prediction_onehot[organ_val-1] * (1 - dissensus[organ_name])
        filtered_groundtruth = consensus[organ_name] * (1 - dissensus[organ_name])
        
        predictions[organ_name] = filtered_prediction
        groundtruth_consensus[organ_name] = filtered_groundtruth
        
        # Compute mean probabilities and confidence in the consensus area
        prob_in_consensus_organ = prob_pred[organ_val-1] * np.where(consensus[organ_name]==1, 1, np.nan)
        prob_in_consensus_bck = prob_pred[organ_val-1] * np.where(consensus[f"{organ_name}_bck"]==1, 1, np.nan)
        mean_conf_organ = np.nanmean(prob_in_consensus_organ)
        mean_conf_bck = np.nanmean(prob_in_consensus_bck)        
        confidence[organ_name] = (((1-mean_conf_bck)+mean_conf_organ)/2)
    
    # Create DiceMetric instance
    dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False, ignore_empty=True)

    dice_scores = {}
    for organ_name in organs.values():
        gt = torch.from_numpy(groundtruth_consensus[organ_name])
        pred = torch.from_numpy(predictions[organ_name])
        dice_metric.reset()
        dice_metric(pred, gt)
        dice_scores[organ_name] = dice_metric.aggregate().item()
    
    return dice_scores, confidence
    

'''
Volume Assessment
'''

def volume_metric(groundtruth, prediction, voxel_proportion=1):
    """
    Calculates the Continuous Ranked Probability Score (CRPS) for each volume class,
    by using the ground truths to create a probabilistic distribution that keeps the
    multirater information of having multiple annotations. 
    
    groundtruth: numpy stack list containing the three ground truths [gt1, gt2, gt3]
                 each gt has the following values: 1: pancreas, 2: kidney, 3: liver
                    (3, slices, X, Y)
    prob_pred: probability prediction matrix, shape: (3, slices, X, Y), the three being
                a probability matrix per each class
    voxel_proportion: vaue of the resampling needed voxel-wise, 1 by default
     
    @output crps_dict
    """
    
    cdf_list = calculate_volumes_distributions(groundtruth, voxel_proportion)
        
    crps_dict = {}    
    organs =  {1: 'panc', 2: 'kidn', 3: 'livr'}

    for organ_val, organ_name in organs.items():
        probabilistic_volume = compute_probabilistic_volume(prediction[organ_val-1], voxel_proportion)
        crps_dict[organ_name] = crps_computation(probabilistic_volume, cdf_list[organ_name], mean_gauss[organ_name], var_gauss[organ_name])

    return crps_dict


def heaviside(x):
    return 0.5 * (np.sign(x) + 1)


def crps_computation(predicted_volume, cdf, mean, std_dev):
    """
    Calculates the Continuous Ranked Probability Score (CRPS) for each volume class,
    by using the ground truths to create a probabilistic distribution that keeps the
    multirater information of having multiple annotations. 
    
    predicted_volume: scalar value representing the volume obtained from the 
                        probabilistic prediction
    cdf: cumulative density distribution (CDF) of the groundtruth volumes
    mean: mean of the gaussian distribution obtained from the three groundtruth volumes
    std_dev: std_dev of the gaussian distribution obtained from the three groundtruth volumes
     
    @output crps_dict
    """
    
    def integrand(y):
        return (cdf(y) - heaviside(y - predicted_volume)) ** 2
    
    lower_limit = mean - 3 * std_dev
    upper_limit = mean + 3 * std_dev
    
    crps_value, _ = quad(integrand, lower_limit, upper_limit)
        
    return crps_value


def calculate_volumes_distributions(groundtruth, voxel_proportion=1):
    """
    Calculates the Cumulative Distribution Function (CDF) of the Probabilistic Function Distribution (PDF)
    obtained by calcuating the mean and the variance of considering the three annotations.
    
    groundtruth: numpy stack list containing the three ground truths [gt1, gt2, gt3]
                 each gt has the following values: 1: pancreas, 2: kidney, 3: liver
                    (3, slices, X, Y)
    voxel_proportion: vaue of the resampling needed voxel-wise, 1 by default
    
    @output cdfs_dict
    """
    
    organs = {1: 'panc', 2: 'kidn', 3: 'livr'}
    
    global mean_gauss, var_gauss, volumes  # Make them global to access in crps
    mean_gauss = {}
    var_gauss = {}
    volumes = {}

    for organ_val, organ_name in organs.items():
        volumes[organ_name] = [np.unique(gt, return_counts=True)[1][organ_val] * np.prod(voxel_proportion) for gt in groundtruth]
        mean_gauss[organ_name] = np.mean(volumes[organ_name])
        var_gauss[organ_name] = np.std(volumes[organ_name])

    # Create normal distribution objects
    gaussian_dists = {organ_name: norm(loc=mean_gauss[organ_name], scale=var_gauss[organ_name]) for organ_name in organs.values()}
    
    # Generate CDFs
    cdfs = {}
    for organ_name in organs.values():
        x = np.linspace(gaussian_dists[organ_name].ppf(0.01), gaussian_dists[organ_name].ppf(0.99), 100)
        cdf_values = gaussian_dists[organ_name].cdf(x)
        cdfs[organ_name] = interp1d(x, cdf_values, bounds_error=False, fill_value=(0, 1))  # Create an interpolation function

    return cdfs
    
    
def compute_probabilistic_volume(preds, voxel_proportion=1):
    """
    Computes the volume of the matrix given (either pancreas, kidney or liver)
    by adding up all the probabilities in this matrix. This way the uncertainty plays
    a role in the computation of the predicted organ. If there is no uncertainty, the 
    volume should be close to the mean obtained by averaging the three annotations.
    
    preds: probabilistic matrix of a specific organ
    voxel_proportion: vaue of the resampling needed voxel-wise, 1 by default
     
    @output volume
    """
    
    # Sum the predicted probabilities to get the volume
    volume = preds.sum().item()
    
    return volume*voxel_proportion


'''
Expected Calibration Error
'''

def multirater_expected_calibration_error(annotations_list, prob_pred):
    """
    Returns a list of length three of the Expected Calibration Error (ECE) per annotation.
    
    annotations_list: list of length three containing the three annotations
    prob_pred: probability prediction matrix, shape: (3, slices, X, Y), the three being
                a probability matrix per each class
     
    @output ece_dict
    """
    
    ece_dict = {}

    for e in range(3):
        ece_dict[e] = expected_calibration_error(annotations_list[e], prob_pred)
        
    return ece_dict


def expected_calibration_error(groundtruth, prob_pred_onehot, num_classes=4, n_bins=50):
    """
    Computes the Expected Calibration Error (ECE) between the given annotation and the 
    probabilistic prediction
    
    groundtruth: groundtruth matrix containing the following values: 1: pancreas, 2: kidney, 3: liver
                    shape: (slices, X, Y)
    prob_pred_onehot: probability prediction matrix, shape: (3, slices, X, Y), the three being
                    a probability matrix per each class
    num_classes: number of classes
    n_bins: number of bins                    
                    
    @output ece
    """ 
    
    # Convert inputs to torch tensors
    all_groundtruth = torch.tensor(groundtruth)
    all_samples = torch.tensor(prob_pred_onehot)
    
    # Calculate the probability for the background class
    background_prob = 1 - all_samples.sum(dim=0, keepdim=True)
    
    # Combine background probabilities with the provided probabilities
    all_samples_with_bg = torch.cat((background_prob, all_samples), dim=0)
    
    # Flatten the tensors to (num_samples, num_classes) and (num_samples,)
    all_groundtruth_flat = all_groundtruth.view(-1)
    all_samples_flat = all_samples_with_bg.permute(1, 2, 3, 0).reshape(-1, num_classes)
    
    # Initialize the calibration error metric
    calibration_error = MulticlassCalibrationError(num_classes=num_classes, n_bins=n_bins)

    # Calculate the ECE
    ece = calibration_error(all_samples_flat, all_groundtruth_flat).cpu().detach().numpy().astype(np.float64)
    
    return ece

def prepare_inputs_for_ace(groundtruth, bin_pred, prob_pred):
    background_prob = 1 - np.sum(prob_pred, axis=0, keepdims=True)
    prob_pred_full = np.concatenate([background_prob, prob_pred], axis=0)

    confids = np.max(prob_pred_full, axis=0)
    flat_pred = bin_pred.flatten()
    flat_gt = groundtruth.flatten()
    flat_conf = confids.flatten()
    correct = (flat_pred == flat_gt).astype(np.int32)

    return correct, flat_conf

def calib_stats(correct, calib_confids):
    n_bins = 20
    y_true = column_or_1d(correct)
    y_prob = column_or_1d(calib_confids)

    if y_prob.min() < 0 or y_prob.max() > 1:
        raise ValueError("y_prob has values outside [0, 1]")

    labels = np.unique(y_true)
    if len(labels) > 2:
        raise ValueError(f"Only binary classification is supported. Provided labels {labels}.")
    y_true = label_binarize(y_true, classes=labels)[:, 0]

    bins = np.linspace(0.0, 1.0 + 1e-8, n_bins + 1)
    binids = np.digitize(y_prob, bins) - 1

    bin_sums = np.bincount(binids, weights=y_prob, minlength=len(bins))
    bin_true = np.bincount(binids, weights=y_true, minlength=len(bins))
    bin_total = np.bincount(binids, minlength=len(bins))

    nonzero = bin_total != 0
    num_nonzero = len(nonzero[nonzero == True])
    prob_true = bin_true[nonzero] / bin_total[nonzero]
    prob_pred = bin_sums[nonzero] / bin_total[nonzero]

    bin_discrepancies = np.abs(prob_true - prob_pred)
    return bin_discrepancies, num_nonzero

def calc_ace(correct, calib_confids):
    bin_discrepancies, num_nonzero = calib_stats(correct, calib_confids)
    return (1 / num_nonzero) * np.sum(bin_discrepancies)




SyntaxError: invalid non-printable character U+00A0 (328903675.py, line 22)

In [ ]:
import pandas as pd
import re

def apply_metrics (paths):
	ct_name="Case_"+re.findall(r'\d+',"path to pred")
	ct_image, annotations, results = format_normalisation("path to prediction.nii.gz", "path to prediction_raw.nii.gz", "path to gt1", "path to gt2", "path to gt3", "path to ct_scan")
	cropped_annotations, cropped_bin_pred, cropped_prob_pred = preprocess_results(ct_image, annotations, results)
	dice_scores, confidence = consensus_dice_score(cropped_annotations, cropped_bin_pred, cropped_prob_pred)
	ece_scores = multirater_expected_calibration_error(cropped_annotations, cropped_prob_pred)
	correct, calib_confids = prepare_inputs_for_ace(np.stack(annotations,axis=0), results[0], np.stack([results[1],results[2],results[3]]))
	ace_score = calc_ace(correct, calib_confids)
	crps_score = volume_metric(np.stack(cropped_annotations, axis=0), cropped_prob_pred)
    	return {"CT": ct_name, "DICE_panc": dice_scores['panc'], "DICE_kidn": dice_scores['kidn'], "DICE_livr": dice_scores['livr'], "ECE_0": ece_scores[0], "ECE_1": ece_scores[1], "ECE_2": ece_scores[2], "ACE": ace_score, "CRPS_panc": crps_score['panc'], "CRPS_kidn": crps_score['kidn'], "CRPS_livr": crps_score['livr']}

df = pd.DataFrame(columns=["CT", "DICE_panc", "DICE_kidn", "DICE_livr", "ECE_0", "ECE_1", "ECE_2", "ACE", "CRPS_panc", "CRPS_kidn", "CRPS_livr"])

for i in paths_list : 
	line=pd.Dataframe(apply_metrics(i))
	df=pd.concat([df,line], ignore_index=True)

In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.utils import column_or_1d
from sklearn.preprocessing import label_binarize
import numpy as np
import nibabel as nib

ModuleNotFoundError: No module named 'nibabel'

In [3]:
img = nib.load('UKCHLL_001_000.nii.gz')

data = img.get_fdata()

print(data.shape) 

NameError: name 'nib' is not defined

In [ ]:
#AUROC
""" y_true=true labels et y_score= score d'incertitude"""
y_true = [1,0,1,1,0,1,0,0,0,1]
y_score = [0.1, 0.4, 0.35, 0.8, 0.7, 0.2, 0.9, 0.6, 0.8, 0.4]
# courbe qui plot les nb de TP sur le nb de val positives (tx de vrai positifs) en fonction du nb de FP sur le nb de val négatives (tx de faux positifs)
#ici utilise des probas --> on fixe un seuil à partir duquel un proba est considérée comme préd positive (par ex, ttes les probas au-dessus de 0.5)
fpr, tpr, _ = roc_curve(y_true, y_score)
#juste l'aire sous la courbe (1=parfait)
roc_auc = auc(fpr, tpr)

print(roc_auc)
#OOD detection rate
""" ils font juste nombre de ood trouvés/nb ood mais la manière dont est défini le OOD est peu claire et semble arbitraire"""

0.16000000000000003


' ils font juste nombre de ood trouvés/nb ood mais la manière dont est défini le OOD est peu claire et semble arbitraire'

In [ ]:
#DICE et IoU
#exemple
y_true = [1, 0, 1, 1, 0, 1, 0]
y_pred = [1, 0, 1, 0, 0, 1, 1]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
#on ignore les TN car très nombreux (faussent le score), score de similarité --> nb de bonnes préd / toutes les éléments de préd et de ground truth
dice=2 * tp / (2 * tp + fp + fn)
#idem que dice mais double pas le coef en haut (plus strict dans les résultats)
iou=tp / (tp + fp + fn)

print(dice,iou)

0.75 0.6


In [ ]:
#ACE
#correct=1 si y_true=y_pred et confid=confiance du modèle
#séparation des probas en bins (0-0.1,0.1-0.2...) pour lesquels on calcule le nb de TP --> proba d'être positif dans ce bin
#-->moyenne empirique sur les bins de la différence entre proba d'être TP dans la préd et proba d'être positif en vrai 
#évalue la performance de la prédiction  moyenne à partir d'une partition de l'échantillon
def calib_stats(correct, calib_confids):
    # calib_confids = np.clip(self.confids, 0, 1)
    n_bins = 20
    y_true = column_or_1d(correct)
    y_prob = column_or_1d(calib_confids)

    if y_prob.min() < 0 or y_prob.max() > 1:
        raise ValueError(
            "y_prob has values outside [0, 1] and normalize is " "set to False."
        )

    labels = np.unique(y_true)
    if len(labels) > 2:
        raise ValueError(
            "Only binary classification is supported. " f"Provided labels {labels}."
        )
    y_true = label_binarize(y_true, classes=labels)[:, 0]

    bins = np.linspace(0.0, 1.0 + 1e-8, n_bins + 1)

    binids = np.digitize(y_prob, bins) - 1

    bin_sums = np.bincount(binids, weights=y_prob, minlength=len(bins))
    bin_true = np.bincount(binids, weights=y_true, minlength=len(bins))
    bin_total = np.bincount(binids, minlength=len(bins))

    nonzero = bin_total != 0
    num_nonzero = len(nonzero[nonzero == True])
    prob_true = bin_true[nonzero] / bin_total[nonzero]
    prob_pred = bin_sums[nonzero] / bin_total[nonzero]
    prob_total = bin_total[nonzero] / bin_total.sum()

    bin_discrepancies = np.abs(prob_true - prob_pred)
    return bin_discrepancies, prob_total, num_nonzero


def calc_ace(correct, calib_confids):
    bin_discrepancies, _, num_nonzero = calib_stats(correct, calib_confids)
    return (1 / num_nonzero) * np.sum(bin_discrepancies)

#exemple
correct= [0, 1, 0, 1, 0, 1, 1, 0, 0, 1]
calib_confids= [0.1, 0.9, 0.4, 0.8, 0.2, 0.6, 0.7, 0.3, 0.4, 0.9]
calc_ace(correct,calib_confids)

np.float64(0.25)

In [ ]:
#NCC
#CC= à quel point deux images mis en décalé sont similaires (à un décalage près)
#normalisedCC=idem mais tient davantage compte de la structure/forme de l'image (mesure indépendante de l'amplitude des valeurs, par exemple une image très lumineuse vs une image très sombre)
#littéralement un score de corrélation entre val image 1 et val images 2 mais on ne retire pas la moyenne (ie somme image 1 * image 2/std(1)std(2))
#ici plutôt que comparer deux images on compare les cartes d'incertitudes entre la Ground Truth et la Pred
def compute_ncc(gt_unc_map: np.array, pred_unc_map: np.array):
    """
    Compute the normalized cross correlation between a ground truth uncertainty and a predicted uncertainty map,
    to determine how similar the maps are.
    :param gt_unc_map: the ground truth uncertainty map based on the rater variability
    :param pred_unc_map: the predicted uncertainty map
    :return: float: the normalized cross correlation between gt and predicted uncertainty map
    """
    mu_gt = np.mean(gt_unc_map)
    mu_pred = np.mean(pred_unc_map)
    sigma_gt = np.std(gt_unc_map, ddof=1)
    sigma_pred = np.std(pred_unc_map, ddof=1)
    gt_norm = gt_unc_map - mu_gt
    pred_norm = pred_unc_map - mu_pred
    prod = np.sum(np.multiply(gt_norm, pred_norm))
    ncc = (1 / (np.size(gt_unc_map) * sigma_gt * sigma_pred)) * prod
    return ncc

#exemple
gt_unc_map = np.array([
    [0.2, 0.3, 0.4, 0.5, 0.6],
    [0.3, 0.3, 0.5, 0.6, 0.7],
    [0.4, 0.4, 0.6, 0.7, 0.8],
    [0.5, 0.6, 0.7, 0.8, 0.9],
    [0.6, 0.7, 0.8, 0.9, 1.0]
])

pred_unc_map = np.array([
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.3, 0.4, 0.5, 0.6, 0.7],
    [0.4, 0.5, 0.6, 0.7, 0.8],
    [0.6, 0.7, 0.8, 0.9, 1.0],
    [0.5, 0.6, 0.7, 0.8, 0.9]
])

compute_ncc(gt_unc_map,pred_unc_map)

np.float64(0.8932699455139714)

In [ ]:
#AURC et EAURC
"""
risk=array avec erreurs/loss pour chaque image (proba erreur)
confids=confiance dans la préd
"""
#tri en fonction de la confiance (croissant)
#somme des erreurs parmi val positives pondéré par nb total
#AURC bas = bien

def rc_curve_stats(
    risks: np.array, confids: np.array
) -> tuple[list[float], list[float], list[float]]:
    coverages = []
    selective_risks = []
    assert (
        len(risks.shape) == 1 and len(confids.shape) == 1 and len(risks) == len(confids)
    )

    n_samples = len(risks)
    idx_sorted = np.argsort(confids)

    coverage = n_samples
    error_sum = sum(risks[idx_sorted])

    coverages.append(coverage / n_samples)
    selective_risks.append(error_sum / n_samples)

    weights = []

    tmp_weight = 0
    for i in range(0, len(idx_sorted) - 1):
        coverage = coverage - 1
        error_sum = error_sum - risks[idx_sorted[i]]
        tmp_weight += 1
        if i == 0 or confids[idx_sorted[i]] != confids[idx_sorted[i - 1]]:
            coverages.append(coverage / n_samples)
            selective_risks.append(error_sum / (n_samples - 1 - i))
            weights.append(tmp_weight / n_samples)
            tmp_weight = 0
    return coverages, selective_risks, weights

def aurc(risks: np.array, confids: np.array):
    _, risks, weights = rc_curve_stats(risks, confids)
    return sum(
        [(risks[i] + risks[i + 1]) * 0.5 * weights[i] for i in range(len(weights))]
    )

def eaurc(risks: np.array, confids: np.array):
    """Compute normalized AURC, i.e. subtract AURC of optimal CSF (given fixed risks)."""
    n = len(risks)
    # optimal confidence sorts risk. Asencding here because we start from coverage 1/n
    selective_risks = np.sort(risks).cumsum() / np.arange(1, n + 1)
    aurc_opt = selective_risks.sum() / n
    return aurc(risks, confids) - aurc_opt


#exemple
confids = np.array([0.9, 0.8, 0.3, 0.6])  

risks = np.array([0, 1, 1, 0]) 
print(aurc(risks,confids))
print(eaurc(risks,confids))

0.2708333333333333
0.0625


In [ ]:
#UNC MAP (mais très emmêlé) --> utile ?
def get_gt_unc_map(self, image_id):
        if self.exp_version.gt_unc_map_loading is None:
            n_reference_segs = self.exp_version.n_reference_segs
            reference_segs_paths = [
                self.ref_seg_dir / f"{image_id}_{i:02d}{self.exp_version.image_ending}"
                for i in range(n_reference_segs)
            ]
            reference_segs = []
            for reference_seg_path in reference_segs_paths:
                reference_seg, _ = load(reference_seg_path)
                reference_segs.append(reference_seg)
            reference_segs = np.array(reference_segs)
            per_pixel_variance = np.var(reference_segs, axis=0)
        else:
            per_pixel_variance = hydra.utils.instantiate(
                self.exp_version.gt_unc_map_loading,
                image_id=image_id,
                dataloader=self.dataloader,
            )
        return per_pixel_variance